#Import libraries

In [ ]:
!pip install tensorflow tensorflow-hub tensorflow-text spacy torch scikit-learn h5py

  Using cached tensorflow_text-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1

In [ ]:
!pip install nlp-id #https://github.com/kumparan/nlp-id for bahasa Indonesia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 26.2 MB/s eta 0:00:00
  Created wheel for nlp-id: filename=nlp_id-0.1.15.0-py3-none-any.whl size=58153892 sha256=1b864701a7468d9c2bd789fcd68152f95e066a9b6c3d0c64878659f1668ff25d
  Stored in directory: /root/.cache/pip/wheels/85/34/2a/deaf7b7896a1eed336b874e1a7732588d40bb7bdc0a824e73e
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=01c8c45b24363b022f7dd544eea1f915d8bd65bf01e3b6463575f765e529a015
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built nlp-id wget
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:
      Successfully uninstalled pytest-7.4.4


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
import nlp_id
import nltk
from nlp_id.stopword import StopWord
from nlp_id.tokenizer import PhraseTokenizer
from nlp_id.postag import PosTag
import h5py
import pickle

# Load Dataset

In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/Semaroam/CapstoneProject-ML/main/dataset/all_tourism_semarang.csv', sep = ',')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Place_Id       57 non-null     int64 
 1   Place_Name     57 non-null     object
 2   Place_Ratings  57 non-null     int64 
 3   Description    57 non-null     object
 4   City           57 non-null     object
 5   Category       57 non-null     object
 6   Image          57 non-null     object
dtypes: int64(2), object(5)
memory usage: 3.2+ KB


In [ ]:
data = dataset.copy()
data.drop(columns='Category', inplace=True)
data

,Place_Id,Place_Name,Place_Ratings,Description,City,Image
0,344,Pantai Marina,2,"Pantai Marina (bahasa Jawa: ꦥꦱꦶꦱꦶꦂ​ꦩꦫꦶꦤ, trans...",Semarang,https://storage.googleapis.com/semaroam-photos...
1,373,Museum Kereta Ambarawa,3,Museum Kereta Api Ambarawa (bahasa Inggris: In...,Semarang,https://storage.googleapis.com/semaroam-photos...
2,336,Grand Maerakaca,4,Masyarakat Jawa Tengah mungkin sudah tidak asi...,Semarang,https://storage.googleapis.com/semaroam-photos...
3,371,Wisata Eling Bening,3,Sebuah fasilitas wisata yang dikelola dengan c...,Semarang,https://storage.googleapis.com/semaroam-photos...
4,389,Brown Canyon,2,"Di Indonesia, Brown Canyon adalah sebuah bekas...",Semarang,https://storage.googleapis.com/semaroam-photos...
5,384,Kampung Tematik Jawi,5,Tak dipungkiri lagi kalau Kampung Jawi Gunungp...,Semarang,https://storage.googleapis.com/semaroam-photos...
6,390,Durian Gardens and Tourism Education Watu Simbar,2,"Perlu diketahui, watu simbar ini merupakan lok...",Semarang,https://storage.googleapis.com/semaroam-photos...
7,367,Wisata Lereng Kelir,2,Wisata yang ada di kawasan gunung Kelir ini me...,Semarang,https://storage.googleapis.com/semaroam-photos...
8,382,Kampoeng Kopi Banaran,2,"Kampoeng Kopi Banaran, sebuah agro wisata perk...",Semarang,https://storage.googleapis.com/semaroam-photos...
9,376,Semarang Chinatown,2,Terbentuknya Pecinan Semarang tak terlepas dar...,Semarang,https://storage.googleapis.com/semaroam-photos...


In [ ]:
labels = dataset['Category']
labels

0            Bahari
1            Budaya
2     Taman Hiburan
3        Cagar Alam
4        Cagar Alam
5            Budaya
6        Cagar Alam
7        Cagar Alam
8     Taman Hiburan
9            Budaya
10           Budaya
11    Taman Hiburan
12    Taman Hiburan
13       Cagar Alam
14    Taman Hiburan
15           Budaya
16       Cagar Alam
17           Budaya
18       Cagar Alam
19           Budaya
20       Cagar Alam
21    Taman Hiburan
22       Cagar Alam
23           Budaya
24       Cagar Alam
25           Budaya
26       Cagar Alam
27    Taman Hiburan
28       Cagar Alam
29    Taman Hiburan
30       Cagar Alam
31       Cagar Alam
32    Taman Hiburan
33           Budaya
34       Cagar Alam
35       Cagar Alam
36    Taman Hiburan
37       Cagar Alam
38           Budaya
39       Cagar Alam
40    Taman Hiburan
41    Tempat Ibadah
42    Taman Hiburan
43    Tempat Ibadah
44    Taman Hiburan
45    Taman Hiburan
46           Budaya
47       Cagar Alam
48    Tempat Ibadah
49           Bahari


#Albert embedding

In [ ]:
# Load ALBERT model and tokenizer from TensorFlow Hub
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_preprocess/3", trainable=False)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/3", trainable=False)

In [ ]:
def get_albert_embeddings(texts):
    preprocessed_texts = preprocessor(texts)
    embeddings = encoder(preprocessed_texts)["pooled_output"]
    return embeddings

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stopword = StopWord()
tokenizer = PhraseTokenizer()

# Preprocess text function using nlp-id
def preprocess_text(text):
    text = stopword.remove_stopword(text)
    return text

def tokenize_text(text):
    return tokenizer.tokenize(text)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.22 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own

In [ ]:
# Encode labels as integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(dataset['Category'])

In [ ]:
dataset['preprocessed_description'] = dataset['Description'].apply(preprocess_text)
dataset['tokens'] = dataset['Description'].apply(tokenize_text)

In [ ]:
for description, tokens in zip(dataset['Description'], dataset['tokens']):
    print(f"Description: {description}")
    print(f"Tokens: {tokens}")
    print()

Description: Pantai Marina (bahasa Jawa: ꦥꦱꦶꦱꦶꦂ​ꦩꦫꦶꦤ, translit. Pasisir Marina) adalah salah satu objek wisata pantai yang berada di kota Semarang, Jawa Tengah. Pantai ini menyimpan pemandangan eksotis khas kota Semarang. Dahulu, tempat ini merupakan hutan bakau dan tambak, tapi pemerintah setempat mengubahnya menjadi tempat rekreasi dengan cara reklamasi daratan. Hasil reklamasi dari hutan bakau ini, sekarang berupa perumahan, pertokoan, dan perkantoran, di sebelah selatan pantai.

Tokens: ['Pantai', 'Marina', '(', 'bahasa', 'Jawa', ':', 'ꦥꦱꦶꦱꦶꦂ\u200bꦩꦫꦶꦤ', ',', 'translit', '.', 'Pasisir Marina', ')', 'adalah salah', 'satu', 'objek', 'wisata', 'pantai', 'yang', 'berada', 'di', 'kota', 'Semarang', ',', 'Jawa Tengah', '.', 'Pantai', 'ini', 'menyimpan', 'pemandangan eksotis', 'khas', 'kota', 'Semarang', '.', 'Dahulu', ',', 'tempat', 'ini', 'merupakan', 'hutan', 'bakau', 'dan', 'tambak', ',', 'tapi', 'pemerintah', 'setempat', 'mengubah', 'nya', 'menjadi', 'tempat', 'rekreasi', 'dengan', '

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(dataset['preprocessed_description'], labels, test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")

Training set size: 33
Validation set size: 12
Test set size: 12


In [ ]:
# Get ALBERT embeddings
train_embeddings = get_albert_embeddings(train_data)
val_embeddings = get_albert_embeddings(val_data)
test_embeddings = get_albert_embeddings(test_data)

In [ ]:
train_embeddings_pt = torch.tensor(train_embeddings.numpy())
val_embeddings_pt = torch.tensor(val_embeddings.numpy())
test_embeddings_pt = torch.tensor(test_embeddings.numpy())



---

#CNN classifier



In [ ]:
class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(128 * ((input_dim - 3 + 1) // 2), num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.pool(F.relu(x))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

input_dim = train_embeddings_pt.size(-1)
num_classes = len(label_encoder.classes_)
classifier = CNNClassifier(input_dim, num_classes)

In [ ]:
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long)

# Training loop
for epoch in range(10):
    classifier.train()
    optimizer.zero_grad()
    outputs = classifier(train_embeddings_pt)
    loss = criterion(outputs, train_labels_tensor)
    loss.backward()
    optimizer.step()

    # Validation
    classifier.eval()
    with torch.no_grad():
        val_outputs = classifier(val_embeddings_pt)
        val_loss = criterion(val_outputs, val_labels_tensor)
        print(f"Epoch {epoch+1}, Validation Loss: {val_loss.item()}")

Epoch 1, Validation Loss: 4.86316442489624
Epoch 2, Validation Loss: 9.97754955291748
Epoch 3, Validation Loss: 7.5107035636901855
Epoch 4, Validation Loss: 8.720458030700684
Epoch 5, Validation Loss: 6.956239223480225
Epoch 6, Validation Loss: 3.443128824234009
Epoch 7, Validation Loss: 8.163866996765137
Epoch 8, Validation Loss: 4.6616315841674805
Epoch 9, Validation Loss: 4.776221752166748
Epoch 10, Validation Loss: 4.436117649078369




---

#NER

In [ ]:
print(dataset['Description'].apply(type).value_counts())

Description
<class 'str'>    57
Name: count, dtype: int64


In [ ]:
# Ensure all values in 'Description' are strings
dataset['Description'] = dataset['Description'].astype(str)

In [ ]:
postagger = PosTag()

def pos_tagging_and_ner(text):
    print(f"Processing text: {text}")  # Debug print
    if not isinstance(text, str):
        raise ValueError(f"Expected a string, got {type(text)}")

    try:
        tokens = tokenizer.tokenize(text)
        print(f"Tokens: {tokens}")  # Debug print
        if not isinstance(tokens, list) or not all(isinstance(t, str) for t in tokens):
            raise ValueError(f"Expected list of strings, got {tokens}")

        # Debug: Verify tokens before passing to POS tagger
        print(f"Token types: {[type(token) for token in tokens]}")

        # Join tokens back into a single string before passing to POS tagger
        combined_tokens = ' '.join(tokens)

        # Check input to POS tagger
        print(f"Tokens being passed to POS tagger: {combined_tokens}")

        pos_tags = postagger.get_phrase_tag(combined_tokens)

        # Check output from POS tagger
        print(f"POS Tags returned: {pos_tags}")

        # Assuming pos_tags is a list of tuples like [(word, tag), ...]
        if not isinstance(pos_tags, list) or not all(isinstance(tag, tuple) and len(tag) == 2 for tag in pos_tags):
            raise ValueError(f"Expected list of tuples for POS tags, got {pos_tags}")

        named_entities = [(word, tag) for word, tag in pos_tags if tag in ['NNP', 'NNPS']]
        return named_entities
    except Exception as e:
        print(f"Error processing text '{text}': {e}")
        return []

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.22 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own

In [ ]:
# Testing with a single example
sample_text = dataset['Description'].iloc[1]
print(pos_tagging_and_ner(sample_text))

Processing text: Museum Kereta Api Ambarawa (bahasa Inggris: Indonesian Railway Museum, Ambarawa) adalah sebuah stasiun kereta api yang sudah dialihfungsikan menjadi sebuah museum serta merupakan museum perkeretaapian pertama di Indonesia. Museum ini memiliki koleksi kereta api yang pernah berjaya pada zamannya. Museum ini secara administratif berada di Desa Panjang, Ambarawa, Semarang. Museum yang terletak pada ketinggian +474,40 meter ini termasuk dalam Daerah Operasi IV Semarang dan dikelola oleh Unit Pusat Pelestarian dan Desain Arsitektur PT Kereta Api Indonesia bekerja sama dengan Pemerintah Provinsi Jawa Tengah.
Tokens: ['Museum Kereta Api Ambarawa', '(', 'bahasa', 'Inggris', ':', 'Indonesian Railway Museum', ',', 'Ambarawa', ')', 'adalah', 'sebuah', 'stasiun', 'kereta', 'api', 'yang', 'sudah', 'dialihfungsikan', 'menjadi', 'sebuah', 'museum', 'serta', 'merupakan', 'museum perkeretaapian pertama', 'di', 'Indonesia', '.', 'Museum', 'ini', 'memiliki', 'koleksi', 'kereta', 'api', '

In [ ]:
# Apply the function to the dataset
try:
    dataset['named_entities'] = dataset['Description'].apply(pos_tagging_and_ner)
    print(dataset['named_entities'])
except Exception as e:
    print(f"Error: {e}")

Processing text: Pantai Marina (bahasa Jawa: ꦥꦱꦶꦱꦶꦂ​ꦩꦫꦶꦤ, translit. Pasisir Marina) adalah salah satu objek wisata pantai yang berada di kota Semarang, Jawa Tengah. Pantai ini menyimpan pemandangan eksotis khas kota Semarang. Dahulu, tempat ini merupakan hutan bakau dan tambak, tapi pemerintah setempat mengubahnya menjadi tempat rekreasi dengan cara reklamasi daratan. Hasil reklamasi dari hutan bakau ini, sekarang berupa perumahan, pertokoan, dan perkantoran, di sebelah selatan pantai.

Tokens: ['Pantai', 'Marina', '(', 'bahasa', 'Jawa', ':', 'ꦥꦱꦶꦱꦶꦂ\u200bꦩꦫꦶꦤ', ',', 'translit', '.', 'Pasisir Marina', ')', 'adalah salah', 'satu', 'objek', 'wisata', 'pantai', 'yang', 'berada', 'di', 'kota', 'Semarang', ',', 'Jawa Tengah', '.', 'Pantai', 'ini', 'menyimpan', 'pemandangan eksotis', 'khas', 'kota', 'Semarang', '.', 'Dahulu', ',', 'tempat', 'ini', 'merupakan', 'hutan', 'bakau', 'dan', 'tambak', ',', 'tapi', 'pemerintah', 'setempat', 'mengubah', 'nya', 'menjadi', 'tempat', 'rekreasi', 'dengan

---
#Recommender

In [ ]:
# Content-Based Recommender System
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(dataset['preprocessed_description'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def recommend(doc_id, num_recommendations=2):
    sim_scores = list(enumerate(cosine_sim[doc_id]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations + 1]
    return [(dataset['Description'][i], score) for i, score in sim_scores]

recommendations = recommend(0)

print("Classification Outputs (Train):", classifier(train_embeddings_pt))
print("Named Entities:", dataset['named_entities'])
print("Recommendations:", recommendations)

Classification Outputs (Train): tensor([[ 8.5342, 22.4788, 11.2969, 13.0234, 14.9500],
        [ 9.7847, 30.7831,  7.8395, 24.5317, 21.3492],
        [ 9.5237, 19.8258, 11.8409, 15.8422, 14.4062],
        [ 5.3890, 20.2915, 13.1356, 10.1824, 18.3477],
        [10.0706, 22.4647, 10.7005, 18.8990, 16.8481],
        [ 8.0994, 19.1669, 13.9079, 11.8133, 14.2461],
        [10.1209, 25.5608,  8.7383, 20.6531, 21.3816],
        [10.5782, 22.0146, 10.6315, 17.1221, 15.1344],
        [11.3403, 21.2532, 11.4942, 18.0157, 14.7449],
        [10.6784, 19.9814, 12.8680, 15.3430, 13.7296],
        [10.7707, 23.8720, 10.0645, 20.7895, 16.5310],
        [ 4.5238, 19.6623, 12.0451,  9.9405, 21.1556],
        [ 9.4775, 19.8855, 12.1226, 15.4907, 14.7405],
        [ 8.5824, 19.7701, 10.4958, 17.4738, 15.8131],
        [ 9.8209, 20.5608, 12.5543, 14.1516, 13.9508],
        [ 8.0399, 18.8246, 14.6566, 11.2417, 14.6863],
        [ 8.9609, 21.1110, 12.2222, 13.3680, 14.6758],
        [10.8043, 27.3726,  9.778

---
#Saving pipeline

In [ ]:
#model weights
torch.save(classifier.state_dict(), 'cnn_classifier_weights.pth')

#tokenizer and label encoder
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

#tf-idf & cosine
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

#albert
preprocessor_path = 'albert_preprocessor'
encoder_path = 'albert_encoder'
tf.saved_model.save(preprocessor, preprocessor_path)
tf.saved_model.save(encoder, encoder_path)

#entire pipeline
with h5py.File('nlp_pipeline.h5', 'w') as hf:
    classifier_weights = pickle.dumps(classifier.state_dict())
    hf.create_dataset('cnn_classifier_weights', data=np.void(classifier_weights))
    hf.create_dataset('tfidf_matrix', data=tfidf_matrix.toarray())
    hf.create_dataset('cosine_similarity', data=cosine_sim)
    tokenizer_bytes = pickle.dumps(tokenizer)
    label_encoder_bytes = pickle.dumps(label_encoder)
    hf.create_dataset('tokenizer', data=np.void(tokenizer_bytes))
    hf.create_dataset('label_encoder', data=np.void(label_encoder_bytes))
    hf.create_dataset('preprocessor_path', data=preprocessor_path.encode('utf-8'))
    hf.create_dataset('encoder_path', data=encoder_path.encode('utf-8'))

In [ ]:
with h5py.File('nlp_pipeline.h5', 'r') as hf:
    classifier_weights = pickle.loads(hf['cnn_classifier_weights'][()])
    classifier.load_state_dict(classifier_weights)
    preprocessor_path = hf['preprocessor_path'][()].decode('utf-8')
    encoder_path = hf['encoder_path'][()].decode('utf-8')
    preprocessor = tf.saved_model.load(preprocessor_path)
    encoder = tf.saved_model.load(encoder_path)
    tfidf_matrix = hf['tfidf_matrix'][:]
    cosine_sim = hf['cosine_similarity'][:]
    tokenizer = pickle.loads(hf['tokenizer'][()])
    label_encoder = pickle.loads(hf['label_encoder'][()])

print("Pipeline saved and loaded successfully.")

Pipeline saved and loaded successfully.


In [ ]:
f = h5py.File('nlp_pipeline.h5')
for key in f.keys():
    print(key)
    print(type(f[key]))

cnn_classifier_weights
<class 'h5py._hl.dataset.Dataset'>
cosine_similarity
<class 'h5py._hl.dataset.Dataset'>
encoder_path
<class 'h5py._hl.dataset.Dataset'>
label_encoder
<class 'h5py._hl.dataset.Dataset'>
preprocessor_path
<class 'h5py._hl.dataset.Dataset'>
tfidf_matrix
<class 'h5py._hl.dataset.Dataset'>
tokenizer
<class 'h5py._hl.dataset.Dataset'>
